In [440]:
import json
from contextlib import nullcontext
from dataclasses import replace
from pickletools import float8
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np
from IPython.display import display
import openpyxl
from json import JSONDecoder


In [441]:
deposit = 20000

url1 = 'https://api.binance.com/api/v3/ticker/price?symbol=BTCRUB'
item1 = requests.get(url1)
BTCRUB = item1.json()['price']

url2 = 'https://api.binance.com/api/v3/ticker/price?symbol=ETHRUB'
item2 = requests.get(url2)
ETHRUB = item2.json()['price']

url3 = 'https://api.binance.com/api/v3/ticker/price?symbol=USDTRUB'
item3 = requests.get(url3)
USDTRUB = item3.json()['price']

url4 = 'https://api.binance.com/api/v3/ticker/price?symbol=BNBRUB'
item4 = requests.get(url4)
BNBRUB = item4.json()['price']

In [442]:
def extract_json_objects(text, decoder=JSONDecoder()):
    """Find JSON objects in text, and yield the decoded JSON data
    Does not attempt to look for JSON arrays, text, or other JSON types outside
    of a parent JSON object.
    """
    pos = 0
    while True:
        match = text.find('{', pos)
        if match == -1:
            break
        try:
            result, index = decoder.raw_decode(text[match:])
            yield result
            pos = match + index
        except ValueError:
            pos = match + 1

In [443]:
headers = {
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8",
    "Cache-Control": "no-cache",
    "Connection": "keep-alive",
    "Content-Length": "123",
    "content-type": "application/json",
    "Host": "p2p.binance.com",
    "Origin": "https://p2p.binance.com",
    "Pragma": "no-cache",
    "TE": "Trailers",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:88.0) Gecko/20100101 Firefox/88.0"
}

headers_bybit = {
    'Accept': 'application/json',
    'Accept-Language': 'ru-RU',
    'Accept-Encoding': 'gzip, deflate, br',
    'Host': 'api2.bybit.com',
    'Origin': 'https://www.bybit.com',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Connection': 'keep-alive',
    'Referer': 'https://www.bybit.com/',
    'Content-Length': '114',
    }

headers_pexpay = {
    'Content-Type': 'application/json',
    'Accept': '*/*',
    'Accept-Language': 'ru',
    'Accept-Encoding': 'gzip, deflate, br',
    'Host': 'www.pexpay.com',
    'Origin': 'https://www.pexpay.com',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15',
    'Connection': 'keep-alive',
    'Content-Length': '161',
    #'x-ui-request-trace:': 'f4e68e34-79da-45e3-8d3c-2dfe0dbffd55',
    'clienttype': 'web',
    'lang': 'en',
    'csrftoken': 'd41d8cd98f00b204e9800998ecf8427e',
    }

paytypes = [
    #'Advcash',
    #'RUBfiatbalance',
    'TinkoffNew',
    #'YandexMoneyNew',
    'QIWI',
    'RosBankNew',
]


In [444]:
for paytype in paytypes:

    coins = {1,2,3,-1}
    method = int
    {
        28,
        9,
        }
    trades = {
        #'buy',
        'sell',
        }
    
    if paytype == 'TinkoffNew':
        method = 28
    if paytype == 'QIWI':
        method = 9
    if paytype == 'RosBankNew':
        method = 69
      
    results_huobi = []
    
    for coin in coins:
        for trade in trades:

            url = f'https://otc-api.trygofast.com/v1/data/trade-market?coinId={coin}&currency=11&tradeType={trade}&currPage=1&payMethod={method}&acceptOrder=-1&country=&blockType=general&online=1&range=0&amount={deposit}&isThumbsUp=false&isMerchant=false&isTraded=false&onlyTradable=false&isFollowed=false'
            page = requests.get(url)
            with open('result.json', 'w', encoding='utf-8') as f:
                for result in extract_json_objects(page.text):
                    json.dump(result, f, ensure_ascii=False, indent=4)

            try:
                [result['data'][0]["price"]]
            except:
                results_huobi = results_huobi + [None]
            else:
                if ([result['data'][0]["tradeMonthTimes"]][0] == 0 or [result['data'][0]["payTerm"]][0] > 15):
                    results_huobi = results_huobi + [result['data'][1]["price"]]
                else:
                    results_huobi = results_huobi + [result['data'][0]["price"]]

    tradetypes = [
                #"SELL", 
                "BUY",
                ]

    assets = [
        #'RUB',
        'BTC',
        'USDT',
        'ETH',
        'BNB',
    ]

    results = []
    trademethod = []
    trade = []
    fiatUnit = []
    assetUnit = []
    results_bybit = []
    results_pexpay = []
    payment = []
    payment_pexpay = []

    if paytype == 'TinkoffNew':
        payment_pexpay = ["Tinkoff"]
    if paytype == 'QIWI':
        payment_pexpay = ["QIWI"]
    if paytype == 'RosBankNew':
        payment_pexpay = ["SBP"]

    for asset in assets:

        data_pexpay = {
            "page":1,
            "rows":10,
            "payTypes":payment_pexpay, #"QIWI", "SBP"
            "classifies":[],
            "asset":asset,
            "transAmount":str(deposit),
            "fiat":"RUB",
            "merchantCheck":False,
            "filter":{"payTypes":[]},
            "tradeType":"BUY"
            }
        
        url_pexpay = 'https://www.pexpay.com/bapi/c2c/v1/friendly/c2c/ad/search'

        r_pexpay = requests.post(url_pexpay, headers=headers_pexpay, json=data_pexpay)
        dt_pexpay = r_pexpay.json()
        #print((json.dumps(dt, indent=4)))

        try:
            [dt_pexpay["data"][0]["adDetailResp"]["price"]]
        except:
            results_pexpay = results_pexpay + None
        else:
            results_pexpay = results_pexpay + [dt_pexpay["data"][0]["adDetailResp"]["price"]]



    for asset in assets:
        if paytype == 'TinkoffNew':
            payment = ["75"]
        if paytype == 'QIWI':
            payment = ["62"]
        if paytype == 'RosBankNew':
            payment = ["14"]

        data_bybit = {
            "userId":"",
            "tokenId":asset,
            "currencyId":"RUB",
            "payment": payment,
            "side":"1", 
            "size":"10",
            "page":"1",
            "amount": str(deposit),
            }
        
        url = 'https://api2.bybit.com/fiat/otc/item/online'

        r = requests.post(url, headers=headers_bybit, json=data_bybit)
        dt_ = r.json()

        try:
            [dt_["result"]["items"][0]["price"]]
        except:
            results_bybit = results_bybit + [None]
        else:
            results_bybit = results_bybit + [dt_["result"]["items"][0]["price"]]


    for asset in assets:
        for tradetype in tradetypes:
            for i in range(1,2):
                for j in range(1,2):

                    data = {
                        "asset": asset,
                        "fiat": "RUB",
                        #"merchantCheck": True,
                        "merchantCheck": False,
                        "page": i,
                        "payTypes": [paytype],
                        #"payTypes": [],
                        #"publisherType": 'merchant',
                        "publisherType": None,
                        "rows": j,
                        "tradeType": tradetype,
                        "transAmount":  str(deposit),
                    }

                    r = requests.post(
                        'https://p2p.binance.com/bapi/c2c/v2/friendly/c2c/adv/search', headers=headers, json=data)
                    dt = r.json()
                    #print((json.dumps(dt, indent=4)))
                    if [dt['data'][0]['adv']['price']] == None:
                        results = results + [None]
                        trademethod = trademethod + ['0']
                        trade = trade + ['0']
                        assetUnit = assetUnit + ['0']
                        fiatUnit = fiatUnit + ['0']
                    else:
                        results = results + [dt['data'][0]['adv']['price']]
                        trademethod = trademethod + [dt['data'][0]['adv']['tradeMethods'][0]['tradeMethodName']]
                        trade = trade + [dt['data'][0]['adv']['tradeType']]
                        assetUnit = assetUnit + [dt['data'][0]['adv']['asset']]
                        fiatUnit = fiatUnit + [dt['data'][0]['adv']['fiatUnit']]

    series = [
              #1, 
              BTCRUB, 
              USDTRUB,
              ETHRUB, 
              BNBRUB,
        ]

    dt_pd = pd.DataFrame({
        "trademethod": paytype,
        "assetUnit": assetUnit,
        "trade": "Покупка крипты",
        "binance": results,
        "huobi": results_huobi,
        "bybit": results_bybit,
        "pexpay": results_pexpay,
        "spot": series,
    })

    dt_pd['binance'] = dt_pd['binance'].astype(float)
    dt_pd['huobi'] = dt_pd['huobi'].astype(float)
    dt_pd['bybit'] = dt_pd['bybit'].astype(float)
    dt_pd['pexpay'] = dt_pd['pexpay'].astype(float)
    dt_pd['spot'] = dt_pd['spot'].astype(float)
    dt_pd['percent_binance'] = np.round(((dt_pd['binance'] - dt_pd['spot'])/(dt_pd['binance']))*100 - 1.2,decimals=2)
    dt_pd['percent_huobi'] = np.round(((dt_pd['huobi'] - dt_pd['spot'])/(dt_pd['huobi']))*100 - 1.2,decimals=2)
    dt_pd['percent_bybit'] = np.round(((dt_pd['bybit'] - dt_pd['spot'])/(dt_pd['bybit']))*100 - 1.2,decimals=2)
    dt_pd['percent_pexpay'] = np.round(((dt_pd['pexpay'] - dt_pd['spot'])/(dt_pd['pexpay']))*100 - 1.2,decimals=2)
    dt_pd['loss_binance'] = - dt_pd['percent_binance'] * deposit / 100
    dt_pd['loss_huobi'] = - dt_pd['percent_huobi'] * deposit / 100
    dt_pd['loss_bybit'] = - dt_pd['percent_bybit'] * deposit / 100
    dt_pd['loss_pexpay'] = - dt_pd['percent_pexpay'] * deposit / 100
    dt_pd['QIWI_deposit_price'] = np.round(1.012 * dt_pd['spot'], decimals=2)
    

    def highlight_percent(val):
        color = 'coral' if val < 2.0 else ' '
        return 'background-color: %s' % color

    display(dt_pd.style
        .highlight_min(color='coral', subset=['percent_binance','percent_huobi','percent_bybit','percent_pexpay'],axis=None)
        .highlight_max(color='#6ac6d0', subset=['loss_binance','loss_huobi','loss_bybit','loss_pexpay'],axis=None)
        .format(precision=2, na_rep='', thousands=" ")
        #.applymap(highlight_percent, subset=['percent_binance'])
        #.applymap(highlight_percent, subset=['percent_huobi'])
        #.applymap(highlight_percent, subset=['percent_bybit'])
        #.applymap(highlight_loss, subset=['good_loss'])
        )
    
    status_0 = dt_pd[['percent_binance','percent_huobi','percent_bybit','percent_pexpay']].min().min()
    status = f"минимальный процент  = {np.round(dt_pd[['percent_binance','percent_huobi','percent_bybit','percent_pexpay']].min().min(),decimals=2)}%"
    loss = np.round(deposit * status_0 / 100, decimals=2)

    columns = {
        'percent_binance',
        'percent_huobi',
        'percent_bybit',
        'percent_pexpay',
    }

    for column in columns:
        row = dt_pd.index[dt_pd[column] == dt_pd[['percent_binance','percent_huobi','percent_bybit','percent_pexpay']].min().min()].tolist()
        if row != []:
            column_ = column.replace('percent_', '')
            price_ = dt_pd[column_][row].values[0]
            active = dt_pd['assetUnit'][row].values[0]
            pay_method = dt_pd['trademethod'][row].values[0]
            text = f'Лучшая покупка - {active} по цене {price_}, платежный метод - {pay_method}, биржа - {column_},  {status}. Loss {loss} / Deposit {deposit}'
            display(text)
    




,trademethod,assetUnit,trade,binance,huobi,bybit,pexpay,spot,percent_binance,percent_huobi,percent_bybit,percent_pexpay,loss_binance,loss_huobi,loss_bybit,loss_pexpay,QIWI_deposit_price
0,TinkoffNew,BTC,Покупка крипты,1 752 900.00,1 750 000.00,1 753 250.00,1 770 756.41,1 728 728.00,0.18,0.02,0.20,1.17,-36.00,-4.00,-40.00,-234.00,1 749 472.74
1,TinkoffNew,USDT,Покупка крипты,78.07,78.20,78.05,78.05,77.12,0.02,0.18,-0.01,-0.01,-4.00,-36.00,2.00,2.00,78.05
2,TinkoffNew,ETH,Покупка крипты,122 900.00,123 500.00,123 488.00,125 244.46,121 210.40,0.17,0.65,0.64,2.02,-34.00,-130.00,-128.00,-404.00,122 664.92
3,TinkoffNew,BNB,Покупка крипты,22 350.00,,,22 948.31,22 086.74,-0.02,,,2.55,4.00,,,-510.00,22 351.78


'Лучшая покупка - BNB по цене 22350.0, платежный метод - TinkoffNew, биржа - binance,  минимальный процент  = -0.02%. Loss -4.0 / Deposit 20000'

,trademethod,assetUnit,trade,binance,huobi,bybit,pexpay,spot,percent_binance,percent_huobi,percent_bybit,percent_pexpay,loss_binance,loss_huobi,loss_bybit,loss_pexpay,QIWI_deposit_price
0,QIWI,BTC,Покупка крипты,1 789 000.00,1 812 107.64,1 824 000.00,1 906 568.40,1 728 728.00,2.17,3.40,4.02,8.13,-434.00,-680.00,-804.00,-1 626.00,1 749 472.74
1,QIWI,USDT,Покупка крипты,78.52,78.92,78.65,79.00,77.12,0.58,1.08,0.75,1.18,-116.00,-216.00,-150.00,-236.00,78.05
2,QIWI,ETH,Покупка крипты,123 500.00,,123 488.00,125 244.46,121 210.40,0.65,,0.64,2.02,-130.00,,-128.00,-404.00,122 664.92
3,QIWI,BNB,Покупка крипты,22 670.90,,,23 013.20,22 086.74,1.38,,,2.83,-276.00,,,-566.00,22 351.78


'Лучшая покупка - USDT по цене 78.52, платежный метод - QIWI, биржа - binance,  минимальный процент  = 0.58%. Loss 116.0 / Deposit 20000'

,trademethod,assetUnit,trade,binance,huobi,bybit,pexpay,spot,percent_binance,percent_huobi,percent_bybit,percent_pexpay,loss_binance,loss_huobi,loss_bybit,loss_pexpay,QIWI_deposit_price
0,RosBankNew,BTC,Покупка крипты,1 750 690.00,1 759 900.00,1 772 000.00,1 766 300.99,1 728 728.00,0.05,0.57,1.24,0.93,-10.00,-114.00,-248.00,-186.00,1 749 472.74
1,RosBankNew,USDT,Покупка крипты,78.07,78.23,78.12,78.04,77.12,0.02,0.22,0.08,-0.02,-4.00,-44.00,-16.00,4.00,78.05
2,RosBankNew,ETH,Покупка крипты,122 400.00,122 940.00,,125 232.59,121 210.40,-0.23,0.21,,2.01,46.00,-42.00,,-402.00,122 664.92
3,RosBankNew,BNB,Покупка крипты,22 327.00,,,22 948.31,22 086.74,-0.12,,,2.55,24.00,,,-510.00,22 351.78


'Лучшая покупка - ETH по цене 122400.0, платежный метод - RosBankNew, биржа - binance,  минимальный процент  = -0.23%. Loss -46.0 / Deposit 20000'